# Generador de resumenes de videos de youtube

#### Cargar librerías necesarias

In [15]:
import os
import csv
import yt_dlp
import openai
import whisper
import requests
import json
import re

#### Configuración de rutas y parámetros, número de videos y directorios de salida
Se configuran las rutas que se utilizarán, además se especifica la cantidad de videos que se van a tomar del csv y se crean los directorios de salida en caso de que no existan

In [16]:
# Configuración de rutas y parámetros
csv_file_path = r'Organismos.csv'
audio_output_dir = r'audios'
text_output_dir = r'audios-texto'
summary_output_dir = r'dataset'

# Máximo de videos a procesar
max_videos = 10

# Crear los directorios de salida si no existen
os.makedirs(audio_output_dir, exist_ok=True)
os.makedirs(text_output_dir, exist_ok=True)
os.makedirs(summary_output_dir, exist_ok=True)

#### Función para sanitizar nombres de archivos
Al hacer la petición al servidor de Llama había problemas con los nombres de los videos, así que hice que los nombres se limpiaran de caracteres especiales para evitar conflictos.

In [17]:
def sanitize_filename(filename):
    filename = re.sub(r'[\\/*?:"<>|]', '_', filename)
    filename = filename.replace(' ', '')
    return filename

#### Descargar el audio de un video de YouTube y transcribir audio a texto
Con ayuda de yt_dlp se descargan los audios de los videos tomando los url de los videos

In [18]:
def download_audio(video_url, output_dir, sanitized_title):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_dir, f'{sanitized_title}.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

def transcribe_audio(file_path):
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result['text']

#### Función para obtener sinopsis del servidor Llama


In [19]:
def get_synopsis_from_llama(text):
    try:
        url = "http://localhost:1234/v1/completions"
        headers = {"Content-Type": "application/json"}
        payload = {
            "model": "llama-3.2-1b-instruct",
            "prompt": f"""
                A continuación, se presenta un documento que puede ser la transcripción de un video convertido a texto. Necesito un resumen detallado que contenga toda la información relevante y que sea claro y organizado

                Resume los puntos clave discutidos en el video.
                Proporciona un breve contexto para cada punto clave.
                Incluye explícitamente información numérica o datos importantes si se mencionan.
                Destaca conclusiones, opiniones o propuestas presentadas en el video.

                Texto original:
                {text}
                Por favor, crea un resumen preciso y fácil de entender, toma en cuenta que se usara para entrenar a un modelo de lenguaje a traves de embeding.
            """,
            "max_tokens": 1000
        }
        print("Enviando solicitud al servidor Llama...")
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Verifica errores HTTP
        response_data = response.json()
        return response_data['choices'][0]['text']
    except requests.exceptions.RequestException as e:
        return f"Error al conectar con el servidor Llama: {e}"

#### Proceso principal del programa
Del archivo csv seleccionado arriba se extrae el URL del video, en este caso toma el numero máximo de videos que se configuró arriba.
Luego obtiene el nombre del video y lo limpia, hace la transcripción con whisper, guarda el resumen que Llama proporciona y lo almacena en una archivo de texto.
Para evitar duplicidad borra el registro del archivo csv y sigue con el siguiente hasta llegar al máximo de videos especificados

In [20]:
# Leer el archivo CSV y procesar los primeros max_videos
with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)

videos_processed = 0
for row in rows:
    if videos_processed >= max_videos:
        break
    video_url = row['Video_Link']
    
    # Obtener el nombre del archivo descargado y limpiarlo
    sanitized_title = sanitize_filename(row['Video_Title'].strip())
    file_name = os.path.join(audio_output_dir, f"{sanitized_title}.mp3")
    
    print(f"Descargando audio de {video_url}...")
    download_audio(video_url, audio_output_dir, sanitized_title)
    
    # Verificar si el archivo de audio se descargó correctamente
    if os.path.exists(file_name):
        print(f"Transcribiendo audio de {file_name}...")
        text = transcribe_audio(file_name)
        
        # Guardar la transcripción en un archivo de texto y limpiarlo
        output_file = os.path.join(text_output_dir, f"{sanitized_title}_transcription.txt")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"Transcripción guardada en {output_file}")
        
        # Obtener sinopsis del texto transcrito
        print(f"Obteniendo sinopsis de {output_file}...")
        synopsis = get_synopsis_from_llama(text)
        
        # Guardar la sinopsis en un archivo de texto y limpiarlo
        summary_file = os.path.join(summary_output_dir, f"{sanitized_title}_summary.txt")
        with open(summary_file, 'w', encoding='utf-8') as f:
            f.write(synopsis)
        print(f"Sinopsis guardada en {summary_file}")
        
        # Eliminar el registro del archivo CSV
        rows.remove(row)
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=row.keys())
            writer.writeheader()
            writer.writerows(rows)
    else:
        print(f"Error: No se encontró el archivo de audio {file_name}")
    
    videos_processed += 1


Descargando audio de https://www.youtube.com/watch?v=y5YGVJGjz5A&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D...
[youtube] Extracting URL: https://www.youtube.com/watch?v=y5YGVJGjz5A&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D
[youtube] y5YGVJGjz5A: Downloading webpage
[youtube] y5YGVJGjz5A: Downloading ios player API JSON
[youtube] y5YGVJGjz5A: Downloading mweb player API JSON
[youtube] y5YGVJGjz5A: Downloading m3u8 information
[info] y5YGVJGjz5A: Downloading 1 format(s): 251
[download] Destination: audios\Senadoapruebadesaparicióndesieteorganismosautónomosysuabsorciónporelgobiernofederal.webm
[download] 100% of    1.87MiB in 00:00:00 at 2.37MiB/s   
[ExtractAudio] Destination: audios\Senadoapruebadesaparicióndesieteorganismosautónomosysuabsorciónporelgobiernofederal.mp3
Deleting original file audios\Senadoapruebadesaparicióndesieteorganismosautónomosysuabsorciónporelgobiernofederal.webm (pass -k to keep)
Transcribiendo audio de audios\Senadoapruebadesaparicióndesieteorganismos

c:\Users\luis2\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Transcripción guardada en audios-texto\Senadoapruebadesaparicióndesieteorganismosautónomosysuabsorciónporelgobiernofederal_transcription.txt
Obteniendo sinopsis de audios-texto\Senadoapruebadesaparicióndesieteorganismosautónomosysuabsorciónporelgobiernofederal_transcription.txt...
Enviando solicitud al servidor Llama...
Sinopsis guardada en dataset\Senadoapruebadesaparicióndesieteorganismosautónomosysuabsorciónporelgobiernofederal_summary.txt
Descargando audio de https://www.youtube.com/watch?v=-LBqNsA2qyc&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D...
[youtube] Extracting URL: https://www.youtube.com/watch?v=-LBqNsA2qyc&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D
[youtube] -LBqNsA2qyc: Downloading webpage
[youtube] -LBqNsA2qyc: Downloading ios player API JSON
[youtube] -LBqNsA2qyc: Downloading mweb player API JSON
[youtube] -LBqNsA2qyc: Downloading m3u8 information
[info] -LBqNsA2qyc: Downloading 1 format(s): 251
[download] Destination: audios\Avalanlaextincióndesieteórganosau

c:\Users\luis2\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Transcripción guardada en audios-texto\Avalanlaextincióndesieteórganosautónomos_CiroGómezLeyva_transcription.txt
Obteniendo sinopsis de audios-texto\Avalanlaextincióndesieteórganosautónomos_CiroGómezLeyva_transcription.txt...
Enviando solicitud al servidor Llama...
Sinopsis guardada en dataset\Avalanlaextincióndesieteórganosautónomos_CiroGómezLeyva_summary.txt
Descargando audio de https://www.youtube.com/watch?v=0TJEfKJwNkI&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D...
[youtube] Extracting URL: https://www.youtube.com/watch?v=0TJEfKJwNkI&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D
[youtube] 0TJEfKJwNkI: Downloading webpage
[youtube] 0TJEfKJwNkI: Downloading ios player API JSON
[youtube] 0TJEfKJwNkI: Downloading mweb player API JSON
[youtube] 0TJEfKJwNkI: Downloading m3u8 information
[info] 0TJEfKJwNkI: Downloading 1 format(s): 251
[download] Destination: audios\SenadoAPRUEBAlaextinciónde7órganosautónomos.webm
[download] 100% of    2.70MiB in 00:00:00 at 3.39MiB/s   
[ExtractAu

...

Transcripción guardada en audios-texto\IFTcalificacomo'retroceso'reformaparadesaparecerórganosautónomos_transcription.txt
Obteniendo sinopsis de audios-texto\IFTcalificacomo'retroceso'reformaparadesaparecerórganosautónomos_transcription.txt...
Enviando solicitud al servidor Llama...
Sinopsis guardada en dataset\IFTcalificacomo'retroceso'reformaparadesaparecerórganosautónomos_summary.txt
Descargando audio de https://www.youtube.com/watch?v=yjn3mmotPS4&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D...
[youtube] Extracting URL: https://www.youtube.com/watch?v=yjn3mmotPS4&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D
[youtube] yjn3mmotPS4: Downloading webpage
[youtube] yjn3mmotPS4: Downloading ios player API JSON
[youtube] yjn3mmotPS4: Downloading mweb player API JSON
[youtube] yjn3mmotPS4: Downloading m3u8 information
[info] yjn3mmotPS4: Downloading 1 format(s): 251
[download] Destination: audios\Desaparecerlosórganosautónomos_PropuestadelPresidente.webm
[download] 100% of   46.28MiB i

c:\Users\luis2\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Transcripción guardada en audios-texto\Desaparecerlosórganosautónomos_PropuestadelPresidente_transcription.txt
Obteniendo sinopsis de audios-texto\Desaparecerlosórganosautónomos_PropuestadelPresidente_transcription.txt...
Enviando solicitud al servidor Llama...
Sinopsis guardada en dataset\Desaparecerlosórganosautónomos_PropuestadelPresidente_summary.txt
Descargando audio de https://www.youtube.com/watch?v=y8SxooSk27U&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D...
[youtube] Extracting URL: https://www.youtube.com/watch?v=y8SxooSk27U&pp=ygUZcmVmb3JtYSBvcmdhbm9zIGF1dG9ub21vcw%3D%3D
[youtube] y8SxooSk27U: Downloading webpage
[youtube] y8SxooSk27U: Downloading ios player API JSON
[youtube] y8SxooSk27U: Downloading mweb player API JSON
[youtube] y8SxooSk27U: Downloading m3u8 information
[info] y8SxooSk27U: Downloading 1 format(s): 251
[download] Destination: audios\Losprosycontrasdelosorganismosautónomos¿Esbuenaideaeliminarlos_.webm
[download] 100% of    2.86MiB in 00:00:01 at 2.57MiB/s

c:\Users\luis2\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Transcripción guardada en audios-texto\Losprosycontrasdelosorganismosautónomos¿Esbuenaideaeliminarlos__transcription.txt
Obteniendo sinopsis de audios-texto\Losprosycontrasdelosorganismosautónomos¿Esbuenaideaeliminarlos__transcription.txt...
Enviando solicitud al servidor Llama...
Sinopsis guardada en dataset\Losprosycontrasdelosorganismosautónomos¿Esbuenaideaeliminarlos__summary.txt
